In [1]:
# pip install mosek

import numpy as np
from generate_perturbed import generate_perturbed
from scipy.spatial import ConvexHull
import mosek.fusion as mf
import mosek.fusion.pythonic      # Provides operators +, -, @, .T, slicing etc.

In [2]:
A0 = np.array([[0, 0, 0, -0.048],
               [1, 0, 0, 0.676],
               [0, 1, 0, -2.21],
               [0, 0, 1, 2.6]])
B0 = np.array([[-0.12], [0.68], [-1.4], [1]])
n_states = 4
n_inputs = 1
A_uncertain = np.array([[False, False, False, True],
                         [False, False, False, True],
                         [False, False, False, True],
                         [False, False, False, True]])
B_uncertain = np.array([[True], [True], [True], [True]])

# A0 = np.array([[0, 0, 1.2],
#                [1, 0, -3.2],
#                [0, 1, 3.1]])
# B0 = np.array([[-0.2], [-0.8], [1]])
# n_states = 3
# n_inputs = 1
# A_uncertain = np.array([[False, False, True],
#                         [False, False, True],
#                         [False, False, True]])
# B_uncertain = np.array([[True], [True], [True]])

n_plants = 256
A_list = generate_perturbed(A0, n_plants, 0.005)
B_list = generate_perturbed(B0, n_plants, 0.005)


In [3]:
n_unc_A = np.sum(A_uncertain)
n_unc_B = np.sum(B_uncertain)
A_unc_entries = np.zeros(n_plants, n_unc_A)
B_unc_entries = np.zeros(n_plants, n_unc_B)
A_unc_entries = [A_list[i][A_uncertain] for i in range(n_plants)]
B_unc_entries = [B_list[i][B_uncertain] for i in range(n_plants)]

In [4]:
hull_A = ConvexHull(A_unc_entries)
A_hull_idx = np.unique(hull_A.vertices) # Extract unique vertices
v_A = [A_unc_entries[i] for i in A_hull_idx]
v_A = np.array(v_A)
n_A = v_A.shape[0]

hull_B = ConvexHull(B_unc_entries)
B_hull_idx = np.unique(hull_B.vertices)
v_B = [B_unc_entries[i] for i in B_hull_idx]
v_B = np.array(v_B)
n_B = v_B.shape[0]

print(f"> Convex hull of A set has {n_A} vertices")
print(f"> Convex hull of B set has {n_B} vertices")
print(f"> This will lead to {n_A*n_B} LMI constraints")

> Convex hull of A set has 50 vertices
> Convex hull of B set has 51 vertices
> This will lead to 2550 LMI constraints


In [5]:
# map back to matrix space

A0 = A_list[0]
A_verts = [A0.copy() for _ in range(n_A)]  
for i in range(n_A):
    A_verts[i][A_uncertain] = v_A[i]

B0 = B_list[0]
B_verts = [B0.copy() for _ in range(n_B)]  
for i in range(n_B):
    B_verts[i][B_uncertain] = v_B[i]

In [6]:
pd_eps = 1e-3

# Create a new model
M = mf.Model("Simultaneous stabilisation LMI")


M = mf.Model("Simultaneous stabilisation LMI")
G = M.variable("G", [n_states, n_states], mf.Domain.unbounded())
L = M.variable("L", [1, n_states], mf.Domain.unbounded())
P_ij = [[M.variable(f"P_{i}_{j}", [n_states, n_states], mf.Domain.inPSDCone()) for j in range(n_B)] for i in range(n_A)]

for i in range(n_A):
    for j in range(n_B):
        # Construct the ij'th LMI constraint
        A, B = A_verts[i], B_verts[j]
        P = P_ij[i][j]
        AG_plus_BL =  A @ G + B @ L
        LMI_ij = mf.Expr.vstack(
            mf.Expr.hstack(P, AG_plus_BL),
            mf.Expr.hstack(AG_plus_BL.T, G + G.T - P)
        )
        pd_defn = pd_eps * np.eye(2 * n_states)
        M.constraint(LMI_ij - pd_defn, mf.Domain.inPSDCone(2 * n_states))

print(f"> Defined {n_A*n_B} LMI constraints")

> Defined 2550 LMI constraints


In [7]:
import sys

# M.setLogHandler(sys.stdout)
M.solve()
G_val = np.array(G.level()).reshape(n_states, n_states)
L_val = np.array(L.level()).reshape(1, n_states)
K = -L_val @ np.linalg.inv(G_val) # K = -L * inv(G)

print(f"> Computed simultaneously stabilising feedback gain, K = {K}")

> Computed simultaneously stabilising feedback gain, K = [[ 0.19086357 -0.36935648  0.34881276  1.85255551]]


In [8]:
from scipy.optimize import minimize
from Jcalcs import compute_worst_case_J

Q = np.eye(n_states)
R = 1
x_0 = None # or something like x_0 = np.array([[0],[0],[1]]) can be used as required
delta_J_tol = 1e-2

assert compute_worst_case_J(K, A_list, B_list, Q, R, x_0) != np.inf
np.seterr(invalid='ignore') # Suppress runtime warnings related to invalid values (J==Inf)
objective      = lambda K: compute_worst_case_J(K.reshape(n_inputs,n_states), A_list, B_list, Q, R, x_0)
disp_current_J = lambda K: print(f"Current worst-case J: {compute_worst_case_J(K.reshape(n_inputs,n_states), A_list, B_list, Q, R, x_0)}")
results = minimize(objective, K.flatten(), method="BFGS", options={"disp": False, "xrtol": delta_J_tol}, callback=disp_current_J)
np.seterr(invalid='warn')
K_optimized = results.x.reshape(n_inputs,n_states)

worst_J = compute_worst_case_J(K, A_list, B_list, Q, R)
worst_J_optimized = compute_worst_case_J(K_optimized, A_list, B_list, Q, R)
print(f"> Worst J using initial K: {worst_J:.4f}")
print(f"> Worst J using optimized K: {worst_J_optimized:.4f}")

Current worst-case J: 89.54212806923687
Current worst-case J: 86.83790020846745
Current worst-case J: 86.72471518675772
Current worst-case J: 86.54984299770051
Current worst-case J: 86.27174668900014
Current worst-case J: 86.11887630825262
Current worst-case J: 86.10894709818974
Current worst-case J: 86.10676718122751
> Worst J using initial K: 91.5793
> Worst J using optimized K: 86.1068
